## Imports

In [2]:
import pandas as pd

### Results

In [13]:
actions = pd.read_csv('./all_action.csv')
actions.head(100)

,Unnamed: 0,action,label
0,0,pommelhorse,0
1,1,bench_pressing,1
2,2,pull_up,2
3,3,jump_jack,3
4,4,situp,4
5,5,front_raise,5
6,6,squat,6
7,7,push_up,7


In [14]:
df = pd.read_csv('./pose_train.csv')
df.head()

,Unnamed: 0,type,name,L1,L2,L3,L4,L5,L6,L7,...,L94,L95,L96,L97,L98,L99,L100,L101,L102,L103
0,0,front_raise,train951.mp4,6,26,70.0,100.0,130.0,163.0,206.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,front_raise,train952.mp4,12,30,53.0,68.0,96.0,108.0,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,pull_up,test1463.mp4,21,39,55.0,75.0,89.0,113.0,125.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,squat,test2340.mp4,185,207,233.0,266.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,front_raise,stu5_11.mp4,180,216,706.0,739.0,790.0,822.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df['type'].unique()

array(['front_raise', 'pull_up', 'squat', 'bench_pressing', 'jump_jack',
       'situp', 'push_up', 'pommelhorse'], dtype=object)

In [16]:
root_directory = "../../data/RepCount/extracted/train"
annotation_file_path = "./pose_train.csv"
action_file_path = "./all_action.csv" 

In [30]:
import os
import pandas as pd

# Load action and video information DataFrames
actions_df = pd.read_csv(action_file_path)
videos_df = pd.read_csv(annotation_file_path)

# Prepare a list to store image paths and related metadata
data = []

# Define the base directory where action folders are stored
base_dir = root_directory

# Iterate over each action folder in the actions DataFrame
for _, action_row in actions_df.iterrows():
    action = action_row['action']
    label = action_row['label']
    
    # Define the path to the action folder
    action_path = os.path.join(base_dir, action)
    
    # Check for the existence of 'salient1' and 'salient2' folders within the action folder
    for salient_folder in ['salient1', 'salient2']:
        salient_path = os.path.join(action_path, salient_folder)
        
        if os.path.isdir(salient_path):  # Check if the salient folder exists
            # Filter videos_df for this specific action
            action_videos_df = videos_df[videos_df['action'] == action]

            # Iterate over each video folder specified in videos_df for this action
            for _, video_row in action_videos_df.iterrows():
                video_name = video_row['name']
                video_path = os.path.join(salient_path, video_name)
                
                if os.path.isdir(video_path):  # Ensure it's a directory
                    # Iterate over each image file in the video folder
                    for image_file in os.listdir(video_path):
                        image_path = os.path.join(video_path, image_file)
                        
                        if os.path.isfile(image_path):  # Ensure it's a file
                            # Append the image path and metadata to the data list
                            salient_value = 0 if salient_folder == 'salient1' else 1
                            data.append({
                                'action': action,
                                'action_label': label,
                                'salient': salient_folder,
                                'label': salient_value + 2*label,
                                'video': video_name,
                                'image_path': image_path
                            })

# Convert the data list to a DataFrame
images_df = pd.DataFrame(data)


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f4a549f61f0>
Traceback (most recent call last):
  File "/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1441, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/multiprocessing/popen_fork.py", line 44, in wait
    if not wait([self.sentinel], timeout):
  File "/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/home/sadat/miniconda3/envs/cowbytes/lib/python3.8/selectors.py", line 415, in se

KeyError: 'action'

In [ ]:
images_df.head()

In [ ]:
import os
import torch
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class ActionImageDataset(Dataset):
    def __init__(self, root_directory, actions_df, transform=None):
        """
        Initializes the ActionImageDataset.

        Args:
            root_directory (str): The base directory where action folders are stored.
            actions_df (str or pd.DataFrame): Path to the CSV file with action metadata or an existing DataFrame. 
                This DataFrame should contain columns such as 'action' (name of the action) and 'label' (action label).
            videos_df (str or pd.DataFrame): Path to the CSV file with video metadata or an existing DataFrame.
                This DataFrame should contain columns such as 'action' (action name) and 'name' (video folder name).
            transform (callable, optional): A torchvision transform or a composition of transforms to apply to the images.
        
        Attributes:
            data (list of dict): A list containing metadata for each image, including paths and labels.
            transform (callable): Transformations to be applied to the images.
        
        Example usage:
            dataset = ActionImageDataset(root_directory='/path/to/root', actions_df='actions.csv', videos_df='videos.csv', transform=transform)
            dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
        """
         
        # Load action and video information DataFrames
        self.actions_df = pd.read_csv(action_file_path)

        # TODO: Use the annotation file for video information later.
        # videos_df = pd.read_csv(annotation_file_path)

        # Prepare a list to store image paths and related metadata
        data = []

        # Define the base directory where action folders are stored
        base_dir = root_directory

        # Iterate over each action folder in the actions DataFrame
        for _, action_row in actions_df.iterrows():
            action = action_row['action']
            label = action_row['label']
            
            # Define the path to the action folder
            action_path = os.path.join(base_dir, action)
            
            # Check for the existence of 'salient1' and 'salient2' folders within the action folder
            for salient_folder in ['salient1', 'salient2']:
                salient_path = os.path.join(action_path, salient_folder)
                
                if os.path.isdir(salient_path):  # Check if the salient folder exists
                    # Iterate over each video folder within the salient folder
                    for video_folder in os.listdir(salient_path):
                        video_path = os.path.join(salient_path, video_folder)
                        
                        if os.path.isdir(video_path):  # Ensure it's a directory
                            # Iterate over each image file in the video folder
                            for image_file in os.listdir(video_path):
                                image_path = os.path.join(video_path, image_file)
                                
                                if os.path.isfile(image_path):  # Ensure it's a file
                                    # Append the image path and metadata to the data list
                                    salient_value = 0 if salient_folder == 'salient1' else 1
                                    data.append({
                                        'action': action,
                                        'action_label': label,
                                        'salient': salient_folder,
                                        'label': salient_value + 2*label,
                                        'video': video_folder,
                                        'image_path': image_path
                                    })

        # Convert the data list to a DataFrame
        self.images_df = pd.DataFrame(data)

    def __len__(self):
        return len(self.images_df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get the image path and label
        img_path = self.images_df.iloc[idx]['image_path']
        label = self.images_df.iloc[idx]['label']
        
        # Load image
        image = Image.open(img_path).convert("RGB")

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        return image, label


torch.Size([32, 3, 224, 224]) tensor([ 9,  9,  8, 14,  9,  5,  0,  5,  7,  5,  6,  1, 12,  1,  0, 13, 13, 10,
         2, 12,  9, 12, 11, 11,  4, 14,  1, 10,  1, 13,  5, 10])
torch.Size([32, 3, 224, 224]) tensor([ 1,  4,  2, 11,  8,  6,  7,  0, 14, 15,  9,  7, 12,  5,  8,  8, 14, 10,
        15,  7,  0, 11, 11,  4,  0,  6,  5, 14,  1, 10, 12, 13])
torch.Size([32, 3, 224, 224]) tensor([ 0, 10, 12,  1, 14,  4, 11, 13, 13, 13,  1,  7, 13, 12,  8,  1,  3,  0,
        13,  4,  5, 11,  5, 13,  0, 14, 12, 14,  0,  3,  9,  4])
torch.Size([32, 3, 224, 224]) tensor([ 8, 10,  1,  9,  5, 10, 12,  8,  5, 12,  0,  3,  1, 10,  5,  0,  7,  0,
        13,  1, 15, 15,  0,  6,  8,  0, 11, 15, 12,  4,  0, 14])
torch.Size([32, 3, 224, 224]) tensor([ 8, 11,  6,  4, 15, 13, 10, 13, 10,  1,  7,  0, 12, 15,  6,  4,  5,  4,
         0, 12,  6, 15,  1, 10, 11,  4,  7,  6,  9,  3, 15, 11])
torch.Size([32, 3, 224, 224]) tensor([12, 12,  6, 12,  1, 13, 15, 12,  2, 15,  1, 10, 15, 14, 12, 12,  3, 13,
         6, 14,

KeyboardInterrupt: 